导入相关模块。

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

from beautify import enlarge_eyes, smooth_skin, squeeze_face, whiten_face
from blend import blend_image, feather_mask
from calibrate import undistort_image
from detect_face import get_face_mask
from preprocess import gaussian_filter, laplacian_sharpen, median_filter

读取内参矩阵和畸变系数。

In [ ]:
params = np.load("../src/calib_params.npz")
K = params["camera_matrix"]
d = params["dist_coeffs"]
print(f"camera_matrix:\n{K}")
print(f"dist_coeffs:\n{d}")

读取原始图像。

In [ ]:
img = cv2.imread("../images/111.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

去畸变。

In [ ]:
img1 = undistort_image(img, K, d)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

预处理，中值滤波+高斯滤波。

In [ ]:
img2 = median_filter(img1, k_size=5)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

img3 = gaussian_filter(img2, k_size=5, sigma=1.0)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

预处理，拉普拉斯锐化。

In [ ]:
img4 = laplacian_sharpen(img3, alpha=1.5)
plt.imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
plt.axis("off")

人脸区域分割。

In [ ]:
face_mask = get_face_mask(img4)
plt.imshow(face_mask, cmap="gray")
plt.axis("off")
plt.show()

美白。

In [ ]:
img5 = whiten_face(img4, face_mask, clip_limit=2.0, tile_grid_size=(8, 8))
plt.imshow(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

瘦脸。

In [ ]:
img6 = squeeze_face(img5, face_mask, gain=0.7)
plt.imshow(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

大眼。

In [ ]:
img7 = enlarge_eyes(img6, face_mask, gain=1.5, edge_low=5, edge_high=150, k_size=5)
plt.imshow(cv2.cvtColor(img7, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

磨皮。

In [ ]:
img8 = smooth_skin(img7, face_mask, diameter=9, sigma_color=75, sigma_space=75)
plt.imshow(cv2.cvtColor(img8, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

与原图像背景羽化融合。

In [ ]:
alpha = feather_mask(face_mask, radius=15)
img9 = blend_image(img8, img, alpha)
plt.imshow(cv2.cvtColor(img9, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()